In [9]:
import tensorflow as tf
import numpy
import scipy
import cPickle as pickle
from keras.datasets import cifar10
from os.path import expanduser

def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W, strides=[1,strides,strides,1], padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def maxpool2d(x,k=2):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')

def create_model(x,weights,biases,dropout):
    x = tf.reshape(x, shape=[-1,32,32,3])
    
    conv1 = conv2d(x,weights['wc1'],biases['bc1'])
    conv1 = maxpool2d(conv1,k=2)

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    weights['wd1'] = initialize_flatten_weights(conv2.get_shape().as_list(), 1024)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    print fc1.get_shape().as_list()

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

def initialize_flatten_weights(input_size, output_size):
    total_size = 1
    for i in range(len(input_size)-1):
        total_size = total_size * input_size[i+1]
    return tf.Variable(tf.random_normal([total_size, output_size]))

def initialize_weights():

    # Store layers weight & bias
    weights = {
        # 5x5 conv, 1 input, 32 outputs
        'wc1': tf.Variable(tf.random_normal([5, 5, 3, 32])),
        # 5x5 conv, 32 inputs, 64 outputs
        'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
        # fully connected, 7*7*64 inputs, 1024 outputs
        #'wd1': tf.Variable(tf.random_normal([12288, 1024])),
        # 1024 inputs, 10 outputs (class prediction)
        'out': tf.Variable(tf.random_normal([1024, n_classes]))
    }

    biases = {
        'bc1': tf.Variable(tf.random_normal([32])),
        'bc2': tf.Variable(tf.random_normal([64])),
        'bd1': tf.Variable(tf.random_normal([1024])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    return weights, biases

def download_cifar():
    reshape_height = 32
    reshape_width = 32
    reshape_channels = 3
    
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    X_train = X_train.transpose(0,2,3,1)
    X_test = X_test.transpose(0,2,3,1)
    
    #X_train = X_train[0:50000,:,:,:]
    #X_test = X_test[0:10000,:,:,:]
    #y_train = y_train[0:50000]
    #y_test = y_test[0:10000]
    
    y_train_vector = numpy.zeros((len(y_train),10))
    y_test_vector = numpy.zeros((len(y_test),10))
    
    for i in range(len(y_train)):
        y_train_vector[i,y_train[i]] = 1
    for i in range(len(y_test)):
        y_test_vector[i,y_test[i]] = 1
        
    X_train_new = numpy.zeros((50000,reshape_height,reshape_width,reshape_channels))
    X_test_new = numpy.zeros((10000,reshape_height,reshape_width,reshape_channels))
        
    for i in range(len(X_train)):
        if i % 100 == 0:
            print "upsampling training image " + str(i)
        X_train_new[i,:,:,:] = scipy.misc.imresize(X_train[i,:,:,:], [reshape_height,reshape_width,reshape_channels])
    
    for i in range(len(X_test)):
        if i % 100 == 0:
            print "upsampling testing image " + str(i)
        X_test_new[i,:,:,:] = scipy.misc.imresize(X_test[i,:,:,:], [reshape_height,reshape_width,reshape_channels])
    
    x = [(X_train_new, y_train_vector), (X_test_new,y_test_vector)]
    
    pickle.dump(x, open(expanduser("~/adversary/data/cifar_data.pkl"),"w"))
    
def load_cifar():
    print "starting to load cifar images"
    x= pickle.load(open(expanduser("~/adversary/data/cifar_data.pkl"),"r"))
    print "loaded cifar images"
    return x

def run_model():
    
    (x_train,y_train), (x_test, y_test) = load_cifar()

    x = tf.placeholder(tf.float32, [None, height, width, channels])
    y = tf.placeholder(tf.float32, [None, n_classes])
    keep_prob = tf.placeholder(tf.float32)

    weights, biases = initialize_weights()
    pred = create_model(x, weights, biases, keep_prob)

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initializing the variables
    init = tf.initialize_all_variables()

    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)
        
        for i in range(epochs):
            step = 1
            # Keep training until reach max iterations
            while step * batch_size < training_iters:
                batch_x, batch_y = x_train[(step-1)*batch_size:step*batch_size], y_train[(step-1)*batch_size:step*batch_size]
                # Run optimization op (backprop)
                sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                               keep_prob: dropout})
                if step % display_step == 0:
                    # Calculate batch loss and accuracy
                    loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                                      y: batch_y,
                                                                      keep_prob: 1.})
                    print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                          "{:.6f}".format(loss) + ", Training Accuracy= " + \
                          "{:.5f}".format(acc))
                step += 1
            print("Optimization Finished!")

            # Calculate accuracy for 256 mnist test images
            print("Testing Accuracy:", \
                sess.run(accuracy, feed_dict={x: x_test,
                                              y: y_test,
                                              keep_prob: 1.}))   


In [2]:
height = 32
width = 32
channels = 3
n_classes = 10
batch_size = 128
dropout = 1.0
learning_rate = 0.001
training_iters = 5000
epochs = 10
display_step = 10

run_model()

KeyboardInterrupt: 

In [10]:
download_cifar()

upsampling training image 0
upsampling training image 100
upsampling training image 200
upsampling training image 300
upsampling training image 400
upsampling training image 500
upsampling training image 600
upsampling training image 700
upsampling training image 800
upsampling training image 900
upsampling training image 1000
upsampling training image 1100
upsampling training image 1200
upsampling training image 1300
upsampling training image 1400
upsampling training image 1500
upsampling training image 1600
upsampling training image 1700
upsampling training image 1800
upsampling training image 1900
upsampling training image 2000
upsampling training image 2100
upsampling training image 2200
upsampling training image 2300
upsampling training image 2400
upsampling training image 2500
upsampling training image 2600
upsampling training image 2700
upsampling training image 2800
upsampling training image 2900
upsampling training image 3000
upsampling training image 3100
upsampling training 

In [11]:
x=load_cifar()

starting to load cifar images
loaded cifar images


In [12]:
x[0][0].shape

(50000, 32, 32, 3)

In [20]:
(X_train, y_train), (X_test, y_test) = x

In [21]:
X_train /= 255

In [27]:
len(X_train)

50000

In [34]:
mean = numpy.zeros((32,32,3))
for i in range(len(X_train)):
    mean = mean + X_train[i,:,:,:]

In [35]:
mean /= len(X_train)

mean

In [36]:
std = numpy.zeros((32,32,3))
for i in range(len(X_train)):
    std = std+ numpy.square(X_train[i,:,:,:] - mean)

In [39]:
std /= len(X_train-1)

In [40]:
std

array([[[ 0.08288108,  0.08174498,  0.09953146],
        [ 0.08070471,  0.0795032 ,  0.09743518],
        [ 0.08025516,  0.07900287,  0.09703562],
        ..., 
        [ 0.08054358,  0.07941339,  0.09736759],
        [ 0.08126576,  0.08013979,  0.09794825],
        [ 0.08236725,  0.08122424,  0.0987739 ]],

       [[ 0.08124393,  0.08009361,  0.09816673],
        [ 0.07941682,  0.07826139,  0.09644693],
        [ 0.07868602,  0.07747892,  0.09570926],
        ..., 
        [ 0.07852704,  0.0774847 ,  0.09567726],
        [ 0.07943762,  0.0783861 ,  0.09648908],
        [ 0.08058333,  0.07945289,  0.09722576]],

       [[ 0.07968873,  0.07855023,  0.09643989],
        [ 0.07764637,  0.0765811 ,  0.09454306],
        [ 0.07658612,  0.07544728,  0.09333259],
        ..., 
        [ 0.07637583,  0.07539928,  0.09340033],
        [ 0.07739339,  0.07645331,  0.09432907],
        [ 0.07877705,  0.07771519,  0.0952246 ]],

       ..., 
       [[ 0.06352715,  0.05913542,  0.06610099],
        

In [42]:
numpy.divide(X_train-mean,std).shape

(50000, 32, 32, 3)

In [46]:
z=(X_train-mean)

In [51]:
X_train[4]

array([[[ 0.66666667,  0.70588235,  0.77647059],
        [ 0.65882353,  0.69803922,  0.76862745],
        [ 0.69411765,  0.7254902 ,  0.79607843],
        ..., 
        [ 0.63529412,  0.70196078,  0.84313725],
        [ 0.61960784,  0.69803922,  0.83921569],
        [ 0.61568627,  0.69411765,  0.83137255]],

       [[ 0.65882353,  0.70980392,  0.77647059],
        [ 0.6745098 ,  0.7254902 ,  0.78823529],
        [ 0.67058824,  0.71764706,  0.78431373],
        ..., 
        [ 0.62352941,  0.69411765,  0.83137255],
        [ 0.61176471,  0.69019608,  0.82745098],
        [ 0.60392157,  0.68235294,  0.81960784]],

       [[ 0.60392157,  0.66666667,  0.72941176],
        [ 0.58431373,  0.64705882,  0.70980392],
        [ 0.50588235,  0.56470588,  0.63529412],
        ..., 
        [ 0.63137255,  0.69803922,  0.83921569],
        [ 0.61568627,  0.69411765,  0.83137255],
        [ 0.60392157,  0.68235294,  0.81960784]],

       ..., 
       [[ 0.29019608,  0.32941176,  0.31372549],
        

In [50]:
z[4]

array([[[ 0.15407553,  0.17232886,  0.25665176],
        [ 0.14846918,  0.16689263,  0.25155459],
        [ 0.18019435,  0.19119153,  0.27615122],
        ..., 
        [ 0.12133718,  0.16742353,  0.32376165],
        [ 0.10826243,  0.1658491 ,  0.32156722],
        [ 0.1052    ,  0.16284863,  0.31414722]],

       [[ 0.1486302 ,  0.17923592,  0.26131718],
        [ 0.16727608,  0.19819945,  0.27660549],
        [ 0.15993373,  0.18754235,  0.2701869 ],
        ..., 
        [ 0.11355867,  0.16416227,  0.31803851],
        [ 0.10458675,  0.1625138 ,  0.31568949],
        [ 0.09742431,  0.15535812,  0.30803514]],

       [[ 0.09519718,  0.13863459,  0.21862196],
        [ 0.07954267,  0.12353929,  0.2038462 ],
        [-0.00185678,  0.0389982 ,  0.12754565],
        ..., 
        [ 0.12466408,  0.17271278,  0.33218047],
        [ 0.11140808,  0.17042541,  0.3252342 ],
        [ 0.0996418 ,  0.15859718,  0.31284878]],

       ..., 
       [[-0.20609302, -0.16452275, -0.13389059],
        